## Preparations

### Getting FAQ document

In [73]:
import requests

In [74]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

### Preparing OpenAI client and RAG

In [3]:
import os

In [8]:
from openai import OpenAI

In [9]:
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    base_url=os.getenv('OPENAI_PROXY_URL'),
)

In [12]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}],
    )
    
    return response.choices[0].message.content

In [76]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    
    return answer

### Prepare Elasticsearch client

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [49]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        },
    },
}

index_name = "course-homework"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-homework'})

In [50]:
from tqdm.auto import tqdm

In [51]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## [Homework questions](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md)

### [Q1. Running Elastic](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q1-running-elastic)

In [53]:
print('Q1:', es_client.info()['version']['build_hash'])

Q1: 42f05b9372a9a4a470db3b52817899b99a76ee73


### [Q2. Indexing the data](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q2-indexing-the-data)

In [55]:
print('Q2: index')

Q2: index


### [Q3. Searching](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q3-searching)

In [57]:
query = 'How do I execute a command in a running docker container?'

In [58]:
def elastic_search(query):
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    },
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    },
                },
            },
        },
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [59]:
print('Q3: Top score is', elastic_search(query)[0])

Q3: Top score is 75.54128


### [Q4. Filtering](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q4-filtering)

In [61]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields",
                    },
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    },
                },
            },
        },
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [62]:
search_results = elastic_search(query)
q4_result = search_results[2]
print('Q4:', q4_result['question'])

Q4: How do I copy files from a different folder into docker container’s working directory?


### [Q5. Building a prompt](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q5-building-a-prompt)

In [64]:
def build_prompt(query, search_results):
    context_template = '''
Q: {question}
A: {text}
'''.strip()
    
    prompt_template = '''
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
'''.strip()

    context = ''
    for doc in search_results:
        context = context + context_template.format(
            question=doc['question'],
            text=doc['text']
        ) + '\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [65]:
prompt = build_prompt(query, search_results)

print('Q5:', len(prompt))

Q5: 1462


### [Q6. Tokens](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#q6-tokens)

In [67]:
import tiktoken

In [68]:
def token_counter(prompt):
    encoding = tiktoken.encoding_for_model('gpt-4o')
    tokens = encoding.encode(prompt)

    return len(tokens)

In [48]:
print('Q6:', token_counter(prompt), 'tokens')

Q6: 322 tokens


### [Bonus: generating the answer](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#bonus-generating-the-answer-ungraded)

In [59]:
rag(query)

'To execute a command in a running Docker container, follow these steps:\n\n1. List the running containers to find the container ID:\n   ```bash\n   docker ps\n   ```\n   \n2. Execute a command in the specific container using its container ID. For example, to start an interactive bash session:\n   ```bash\n   docker exec -it <container-id> bash\n   ```'

### [Bonus: calculating the costs](https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/01-intro/homework.md#bonus-calculating-the-costs-ungraded)

In [72]:
result = (150 * 5 + 250 * 15) / 1_000_000 * 1_000

print(f'Total cost: {result}$')

Total cost: 4.5$
